In [6]:
import os
from openai import OpenAI
from pinecone import Pinecone

# === 환경변수 읽기 ===
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")

# === 환경변수 검증 (중요!) ===
if not OPENAI_API_KEY:
    raise ValueError("❌ OPENAI_API_KEY가 설정되지 않았습니다.")
if not PINECONE_API_KEY:
    raise ValueError("❌ PINECONE_API_KEY가 설정되지 않았습니다.")
if not PINECONE_INDEX_NAME:
    raise ValueError("❌ PINECONE_INDEX_NAME가 설정되지 않았습니다 (.env 확인).")

# === 클라이언트 생성 ===
client = OpenAI(api_key=OPENAI_API_KEY)
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(PINECONE_INDEX_NAME)

# === 모델 설정 ===
EMBED_MODEL = "text-embedding-3-large"   # 3072-dim
CHAT_MODEL = "gpt-4.1-mini"

print("✅ OpenAI & Pinecone 연결 완료")
print("   - Index:", PINECONE_INDEX_NAME)

✅ OpenAI & Pinecone 연결 완료
   - Index: realestate


In [8]:
def embed_query(text: str):
    res = client.embeddings.create(
        model=EMBED_MODEL,
        input=text
    )
    return res.data[0].embedding


def search_docs(query: str, top_k=8, min_score=0.45):
    v = embed_query(query)
    res = index.query(
        vector=v,
        top_k=top_k,
        include_metadata=True
    )
    return [m for m in res["matches"] if m["score"] >= min_score]

In [16]:
RAG_RULES = """
너는 오직 제공된 [근거]만으로 답한다.
- 근거에 없는 사실/법리/절차/판례/조문을 절대 추가하지 마라.
- 결론은 근거 문장을 최대한 그대로 재구성해라(새 표현 최소화).
- 반드시 근거 문장을 따옴표로 2~3개 인용해라.
- 불확실하면 '근거 부족'이라고 말해라.

"""

def answer_with_rag(query: str, return_matches=False):
    matches = search_docs(query)
    if not matches:
        return ("근거 부족: 관련 문서를 찾지 못했습니다.", matches) if return_matches else "근거 부족: 관련 문서를 찾지 못했습니다."

    context = "\n\n".join(
        f"[근거 {i+1} | score={m['score']:.3f}] {m['metadata'].get('text','')}"
        for i, m in enumerate(matches)
    )

    prompt = f"""{RAG_RULES}



[근거]
{context}

[질문]
{query}

[출력 형식]
1) 결론
2) 근거 인용(따옴표 2~3개)
3) 요약(3줄 이내)
4) 근거 부족한 부분(있으면)
"""

    res = client.chat.completions.create(
        model=CHAT_MODEL,
        temperature=0,
        messages=[{"role": "user", "content": prompt}]
    )
    return res.choices[0].message.content

In [17]:
print(answer_with_rag(
    "임대인이 전세사기 재판 중이거나 주택에 압류·가압류가 있거나 경매 진행 중이면 계약 해지가 가능한가?"
))

1) 결론  
임대인이 전세사기 재판 중이거나 주택에 압류·가압류가 있거나 경매 진행 중인 경우, 임대차계약 해지는 일반적으로 어렵고, 해지권이 발생한다고 볼 수 없으나, 임차인이 전세사기 피해자로 결정되면 신뢰관계 파괴로 계약 해지가 가능할 수 있으며, 경매 진행 시 임차인이 권리신고와 배당요구를 하면 임대차계약 해지와 같은 효력이 발생합니다.

2) 근거 인용  
"임대인의 파산이나 회생신청 등, 가압류, 압류, 근저당권설정 등과 같은 사유를 사정변경에 의한 해지사유로 보기는 어려우므로 임대차계약이 종료되려면 기간만료를 기다려야 하는 것이 일반적입니다."  
"임차인 A의 경우와 같이 ①임대인이 전세사기 가해자가 되어 형사소추를 받고 있는 경우, 아직 이에 관한 사례는 없으나 임차인이 전세사기 피해자로 결정이 된다면 신뢰관계 파괴로 보아 임대차계약을 해지할 수 있을 가능성이 있으며,"  
"다만 본 건물이 경·공매진행시 임차인이 권리신고와 배당요구를 하면 임대차계약을 해지한 것과 같은 효력이 있습니다(대법원 94다37646 판결)."

3) 요약  
- 임대인의 압류·가압류 등은 해지사유로 보기 어렵다.  
- 임대인이 전세사기 가해자로 형사재판 중이라도 해지 가능성은 피해자 결정 시에 한정된다.  
- 경매 진행 시 권리신고와 배당요구를 하면 해지와 같은 효력이 발생한다.

4) 근거 부족한 부분  
임대인이 전세사기 재판 중인 경우 해지 가능성에 대한 구체적 판례나 명확한 법리 근거는 부족하다.


In [12]:
matches = search_docs("임대차계약 해지 사유")
for m in matches[:3]:
    print("score:", m["score"])
    print(m["metadata"]["text"][:300])
    print("-"*60)

score: 0.594979346
2000다69026 판결) 양도인과 양수인 모두가 보증금반환의무가 있고, 임
62
차인의 대항력과 우선변제권도 그대로 유지된다할 것입니다.
그리하여 임차인 A나 B는 소유권이 이전된 사실을 알게 된 날로부터 상당한 기간내에 이
의를 제기하면 임대인 전소유자는 보증금반환 채무가 있고, 현소유자(C)법인은 임차보증금
을 변제할 채무를 전소유자와 병존적으로 부담하는 것이므로, 전소유자와 C법인을 상대로
보증금반환소송을 제기하면 될 것입니다.
(참고로 임대인의 승계사실에 이의를 제기하면 현소유자는 보증금반환 책임이 없고 전소유자만 책
임을
------------------------------------------------------------
score: 0.576351225
(참고로 임대인의 승계사실에 이의를 제기하면 현소유자는 보증금반환 책임이 없고 전소유자만 책 임을 부담한다는 하급심 판결이 있으나, 변론주의 원칙상 주택매매시 보증금 공제사실과 병존적 채 무인수를 주장·입증하면 전소유자와 현소유자가 공동책임을 부담할 것입니다.) * 채무인수 의 종류는 면책적채무인수와 병존적 채무인수가 있습니다. 1. 면책적 채무인수란채무를 인수함에 있어서 종전 채무자는 채무를 면하고 인수인만이 채무를 부담하는 채무인수 방법인데, 이때에는 반드시 채권자의 승낙이 있어야만 면책적 채무인수가 될 수 있습니다. 2. **
------------------------------------------------------------
score: 0.575548232
임대인이 제1항제8호의 사유로 갱신을 거절하였음에도 불구하고 갱신요구가 거절되지 아니하였더라면 갱신되었을 기간이 만료되기 전에 정당한 사유 없이 제3자에게 목적 주택을 임대한 경우 임대인은 갱신거절로 인하여 임차인이 입은 손해를 배상하여야 한다.
------------------------------------------------------------


In [14]:
# pip install -U python-dotenv openai pinecone langchain-text-splitters

In [13]:
import os
from dotenv import load_dotenv

from openai import OpenAI
from pinecone import Pinecone

from langchain_text_splitters import RecursiveCharacterTextSplitter


# ---------------------------
# 0) 환경 로드
# ---------------------------
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")

if not OPENAI_API_KEY or not PINECONE_API_KEY or not PINECONE_INDEX_NAME:
    raise ValueError("OPENAI_API_KEY / PINECONE_API_KEY / PINECONE_INDEX_NAME 를 .env에 설정해줘.")

client = OpenAI(api_key=OPENAI_API_KEY)

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(PINECONE_INDEX_NAME)


# ---------------------------
# 1) 임베딩 유틸
# ---------------------------
def embed_text(text: str) -> list[float]:
    resp = client.embeddings.create(
        model="text-embedding-3-large",  # ✅ 3072 dim
        input=text
    )
    return resp.data[0].embedding


# ---------------------------
# 2) 문서 -> 청크 -> 업서트
# ---------------------------
def upsert_document(
    doc_id: str,
    text: str,
    namespace: str = "default",
    chunk_size: int = 800,
    chunk_overlap: int = 120,
):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""],
    )
    chunks = splitter.split_text(text)

    vectors = []
    for i, chunk in enumerate(chunks):
        vec = embed_text(chunk)
        vectors.append({
            "id": f"{doc_id}::chunk{i}",
            "values": vec,
            "metadata": {
                "doc_id": doc_id,
                "chunk_id": i,
                "text": chunk,
            }
        })

    # 배치 업서트
    index.upsert(vectors=vectors, namespace=namespace)
    return {"doc_id": doc_id, "chunks": len(chunks)}


# ---------------------------
# 3) 검색
# ---------------------------
def retrieve(
    query: str,
    namespace: str = "default",
    top_k: int = 5,
):
    qvec = embed_text(query)
    res = index.query(
        vector=qvec,
        top_k=top_k,
        include_metadata=True,
        namespace=namespace,
    )

    contexts = []
    for m in res.matches or []:
        md = m.metadata or {}
        contexts.append({
            "score": m.score,
            "doc_id": md.get("doc_id"),
            "chunk_id": md.get("chunk_id"),
            "text": md.get("text", ""),
        })
    return contexts


# ---------------------------
# 4) LLM 응답 (RAG)
# ---------------------------
SYSTEM = """너는 계약서/정책 문서를 검토하는 실무형 어시스턴트다.
주어진 근거(context) 안에서만 답하고, 근거가 부족하면 '근거 부족'이라고 말한다.
출력은 한국어로, 핵심 리스크/근거/권고조치 형태로 정리한다.
"""

def answer_with_rag(
    question: str,
    namespace: str = "default",
    top_k: int = 5,
    model: str = "gpt-4o-mini",
):
    ctxs = retrieve(question, namespace=namespace, top_k=top_k)

    context_block = "\n\n".join(
        [f"[doc={c['doc_id']} chunk={c['chunk_id']} score={c['score']:.3f}]\n{c['text']}"
         for c in ctxs]
    ) or "(검색 결과 없음)"

    prompt = f"""아래 context를 근거로 질문에 답해줘.

[context]
{context_block}

[question]
{question}

[output format]
- 리스크 요약(한 줄)
- 주요 리스크(불릿 3~7개)
  - 근거: (doc/chunk 인용)
  - 영향
  - 권고 조치
- 추가로 확인할 질문(있다면)
"""

    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": SYSTEM},
            {"role": "user", "content": prompt},
        ],
        temperature=0.2,
    )

    return {
        "contexts": ctxs,
        "answer": resp.choices[0].message.content
    }


# ---------------------------
# 5) 데모 실행
# ---------------------------
if __name__ == "__main__":
    sample_doc = """
    본 계약의 손해배상 한도는 연간 계약 금액의 100%로 한다.
    단, 고의 또는 중과실, 비밀유지 위반, 지식재산권 침해의 경우 손해배상 한도를 적용하지 않는다.
    계약 해지는 30일 전 서면 통지로 가능하다.
    """

    print("1) 업서트 중...")
    print(upsert_document(doc_id="contract_v1", text=sample_doc, namespace="contracts"))

    q = "손해배상 한도 조항에서 우리에게 불리한 리스크가 뭐야?"
    print("\n2) RAG 답변:\n")
    out = answer_with_rag(q, namespace="contracts", top_k=5)
    print(out["answer"])


1) 업서트 중...
{'doc_id': 'contract_v1', 'chunks': 1}

2) RAG 답변:

- 리스크 요약: 손해배상 한도가 연간 계약 금액의 100%로 제한되어 있어, 특정 상황에서 무제한 책임을 질 수 있다.

- 주요 리스크
  - 근거: "본 계약의 손해배상 한도는 연간 계약 금액의 100%로 한다."
  - 영향: 계약 금액을 초과하는 손해가 발생할 경우, 추가적인 손해에 대한 배상이 불가능해 재정적 부담이 커질 수 있다.
  - 권고 조치: 손해배상 한도를 명확히 하고, 고의 또는 중과실, 비밀유지 위반, 지식재산권 침해와 같은 상황에 대한 책임 범위를 구체적으로 정의하는 조항을 추가하는 것이 필요하다.

- 추가로 확인할 질문: 고의 또는 중과실의 정의와 판단 기준은 무엇인가?
